## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [2]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,27.32,76,80,4.25,broken clouds
1,1,Chumikan,RU,54.7183,135.3150,-23.69,93,98,4.80,overcast clouds
2,2,Kapaa,US,22.0752,-159.3190,25.18,84,75,5.66,light rain
3,3,Hermanus,ZA,-34.4187,19.2345,18.28,73,85,5.99,overcast clouds
4,4,Vaini,TO,-21.2000,-175.2000,25.09,83,75,3.60,broken clouds


In [3]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 

#Ask the customer to add a minimum and maximum temperature value
min_temp = float(input("What is the minimum temperature you would like for your trip, in degrees Celcius? "))
max_temp = float(input("That is the maximum temperature you would like for your trip, in degrees Celcius? "))

What is the minimum temperature you would like for your trip, in degrees Celcius? 25
That is the maximum temperature you would like for your trip, in degrees Celcius? 35


In [4]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.

#Filter the dataset to find cities that fit the criteria
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) & 
                                      (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Butaritari,KI,3.0707,172.7902,27.32,76,80,4.25,broken clouds
2,2,Kapaa,US,22.0752,-159.3190,25.18,84,75,5.66,light rain
4,4,Vaini,TO,-21.2000,-175.2000,25.09,83,75,3.60,broken clouds
10,10,Grand Gaube,MU,-20.0064,57.6608,25.22,72,8,1.34,clear sky
12,12,Atuona,PF,-9.8000,-139.0333,25.78,78,35,10.57,light rain
13,13,Artur Nogueira,BR,-22.5731,-47.1725,30.69,36,15,3.23,few clouds
17,17,Padang,ID,-0.9492,100.3543,26.03,86,22,1.29,few clouds
18,18,Avarua,CK,-21.2078,-159.7750,25.03,94,40,3.60,light rain
22,22,Georgetown,MY,5.4112,100.3354,26.96,92,20,1.54,heavy intensity rain
25,25,Ndele,CF,8.4109,20.6473,29.55,28,76,1.94,broken clouds


In [5]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.count()

City_ID                198
City                   198
Country                197
Lat                    198
Lng                    198
Max Temp               198
Humidity               198
Cloudiness             198
Wind Speed             198
Current Description    198
dtype: int64

In [6]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
preferred_cities_df.replace("", "NaN")
clean_df = preferred_cities_df.dropna()
clean_df.count()

City_ID                197
City                   197
Country                197
Lat                    197
Lng                    197
Max Temp               197
Humidity               197
Cloudiness             197
Wind Speed             197
Current Description    197
dtype: int64

In [7]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,27.32,broken clouds,3.0707,172.7902,
2,Kapaa,US,25.18,light rain,22.0752,-159.3190,
4,Vaini,TO,25.09,broken clouds,-21.2000,-175.2000,
10,Grand Gaube,MU,25.22,clear sky,-20.0064,57.6608,
12,Atuona,PF,25.78,light rain,-9.8000,-139.0333,
13,Artur Nogueira,BR,30.69,few clouds,-22.5731,-47.1725,
17,Padang,ID,26.03,few clouds,-0.9492,100.3543,
18,Avarua,CK,25.03,light rain,-21.2078,-159.7750,
22,Georgetown,MY,26.96,heavy intensity rain,5.4112,100.3354,
25,Ndele,CF,29.55,broken clouds,8.4109,20.6473,


In [8]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    params["location"] = f'{lat},{lng}'
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found...skipping")        

Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping
Hotel not found...skipping


In [9]:
#check for blank "Hotel Name" values 
hotel_df["Hotel Name"].value_counts()

                                      24
Le Rocher de Tahiti                    1
สถานที่จัดงานแต่ง SOFEE & DORN         1
Sylvia Hotel                           1
Clevers Hotel Chiredzi                 1
                                      ..
Shandrani Beachcomber Resort & Spa     1
Hotel Campos                           1
Heritage Park Hotel                    1
Oumou Elghoura                         1
Hostal Jardin Del Sol                  1
Name: Hotel Name, Length: 174, dtype: int64

In [10]:
# 7. Drop the rows where there is no Hotel Name.
clean_hotel_df = hotel_df.loc[(hotel_df["Hotel Name"]!='')]
clean_hotel_df["Hotel Name"].value_counts()


Gîte kajeon                           1
Le Rocher de Tahiti                   1
สถานที่จัดงานแต่ง SOFEE & DORN        1
Sylvia Hotel                          1
Clevers Hotel Chiredzi                1
                                     ..
Shandrani Beachcomber Resort & Spa    1
Hotel Campos                          1
Heritage Park Hotel                   1
Oumou Elghoura                        1
Hostal Jardin Del Sol                 1
Name: Hotel Name, Length: 173, dtype: int64

In [11]:
clean_hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
0,Butaritari,KI,27.32,broken clouds,3.0707,172.7902,Isles Sunset Lodge
2,Kapaa,US,25.18,light rain,22.0752,-159.3190,Sheraton Kauai Coconut Beach Resort
4,Vaini,TO,25.09,broken clouds,-21.2000,-175.2000,Keleti Beach Resort
10,Grand Gaube,MU,25.22,clear sky,-20.0064,57.6608,Veranda Paul et Virginie Hotel & Spa
12,Atuona,PF,25.78,light rain,-9.8000,-139.0333,Villa Enata
13,Artur Nogueira,BR,30.69,few clouds,-22.5731,-47.1725,Novo Hotel
17,Padang,ID,26.03,few clouds,-0.9492,100.3543,Mercure Padang
18,Avarua,CK,25.03,light rain,-21.2078,-159.7750,Paradise Inn
22,Georgetown,MY,26.96,heavy intensity rain,5.4112,100.3354,Cititel Penang
26,Santo Antonio Do Taua,BR,29.97,overcast clouds,-1.1519,-48.1294,Pousada Magno


In [12]:
# 8a. Create the output File (CSV)
output_data_file = 'WeatherPy_vacation.csv'
# 8b. Export the City_Data into a csv
clean_hotel_df.to_csv(output_data_file, index_label="City_ID")

In [13]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °C</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
clean_hotel_info = [info_box_template.format(**row) for index, row in clean_hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = clean_hotel_df[["Lat", "Lng"]]

In [14]:
# 11a. Add a marker layer for each city to the map. 
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
marker_layer = gmaps.marker_layer(locations, info_box_content=clean_hotel_info)
fig.add_layer(marker_layer)
# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))